In [1]:
# -*- coding: utf-8 -*-

#from __future__ import unicode_literals

'''
 * Focusing Neuron Test Case
 *
 * Code Authors:  F. Boray Tek and 
 * İlker Çam contributed to an earlier version
 *
 * All rights reserved.
 *
 * For details, see the paper:
 * 
 *  
 * 
 *
 * Permission to use, copy, modify, and distribute this software and
 * its documentation for educational, research, and non-commercial
 * purposes, without fee and without a signed licensing agreement, is
 * hereby granted, provided that the above copyright notice and this
 * paragraph appear in all copies modifications, and distributions.
 *
 * NOTE: THIS WORK IS PATENT PENDING!
 * Patent rights are owned by F. Boray Tek, İlker Çam, Işık University
 * Any commercial use or any redistribution of this software
 * requires a license from one of the above mentioned establishments.
 *
 * For further details, contact F. Boray Tek (boraytek@gmail.com).
'''

'\n * Focusing Neuron Test Case\n *\n * Code Authors:  F. Boray Tek and \n * İlker Çam contributed to an earlier version\n *\n * All rights reserved.\n *\n * For details, see the paper:\n * \n *  \n * \n *\n * Permission to use, copy, modify, and distribute this software and\n * its documentation for educational, research, and non-commercial\n * purposes, without fee and without a signed licensing agreement, is\n * hereby granted, provided that the above copyright notice and this\n * paragraph appear in all copies modifications, and distributions.\n *\n * NOTE: THIS WORK IS PATENT PENDING!\n * Patent rights are owned by F. Boray Tek, İlker Çam, Işık University\n * Any commercial use or any redistribution of this software\n * requires a license from one of the above mentioned establishments.\n *\n * For further details, contact F. Boray Tek (boraytek@gmail.com).\n'

In [12]:
import os
os.environ['THEANO_FLAGS']='device=cuda0,floatX=float32,preallocate=0.1'
os.environ['MKL_THREADING_LAYER']='GNU'
import sys
sys.path.insert(0, "..")
import time
import matplotlib.pyplot as plt
import lasagne
import theano
theano.config.exception_verbosity = 'high'
import theano.tensor as T
import numpy as np
from focusing import FocusedLayer1D
from collections import OrderedDict
from lasagne.updates import get_or_compute_grads, apply_momentum, sgd
from lasagne.updates import momentum, adam, adadelta
from lasagne.regularization import regularize_layer_params_weighted, l2, l1
from lasagne.regularization import regularize_layer_params



from lasagne_utils import sgdWithLrsClip,categorical_focal_loss,\
get_shared_by_pattern,sgdWithLrs,iterate_minibatches,set_params_value,\
print_param_stats, get_params_values_wkey,sgdWithWeightSupress

from data_utils import load_dataset_mnist, load_dataset_mnist_cluttered,\
load_dataset_fashion, load_dataset_cifar10

import pandas as pd
#pandas.DataFrame(data=None, index=None, columns=None, dtype=None, copy=False)[source]¶
from data_utils import load_blob

In [ ]:
%matplotlib inline
import matplotlib
font = {'family' : 'arial',
        'weight' : 'normal',
        'size'   : 22}

matplotlib.rc('font', **font)

In [ ]:
def build_dense_model(input_shape,input_var, fparams):
    # Initers, Layers
    nhidden = fparams['nHIDDEN']
    nclasses = fparams['nCLASSES']
    batchnorm = fparams['USEBATCHNORM']
    use_penalty = fparams['USE_PENALTY']
    ini = lasagne.init.GlorotUniform()
    
    nonlin = lasagne.nonlinearities.rectify
    softmax = lasagne.nonlinearities.softmax
    
    lin = lasagne.nonlinearities.linear
    
    # Input Layer
    l_in = lasagne.layers.InputLayer(shape=input_shape, input_var=input_var)
    
    # Denses
    l_dense1 = lasagne.layers.DenseLayer(
            l_in, num_units=nhidden, 
            nonlinearity=lin, 
            W=ini, name="dense1")
    
    
    if batchnorm:
    # if you close BATCHNORM weights get LARGE
        l_bn = lasagne.layers.NonlinearityLayer(
                lasagne.layers.BatchNormLayer(l_dense1), nonlinearity=nonlin)
    
    else:
    
        l_bn = lasagne.layers.NonlinearityLayer(l_dense1, nonlinearity=nonlin)
    #l_dense2 = lasagne.layers.DenseLayer(l_dense1, num_units=4, nonlinearity=lasagne.nonlinearities.tanh, W=ini, name='dense2')
    
    #l_drop1 = lasagne.layers.dropout(l_bn, p=0.1)
    
    # Output Layer
    l_out = lasagne.layers.DenseLayer(l_bn, num_units=nclasses, nonlinearity=softmax, W=ini, name='output')
    
    
    penalty = l2(l_dense1.W)*1e-5+l2(l_out.W)*1e-5
    if not use_penalty:
        penalty = penalty*0
    
    #penalty = penalty*0
    #penalty = (l2(l_dense1.W)*1e-30)#(l2(l_dense1.W)*1e-3)+(l1(l_dense1.W)*1e-6) +(l2(l_out.W)*1e-3)
    
    return l_out, penalty

In [ ]:
def build_focused_model(input_shape,input_var, fparams):
    # Initers, Layers
    
    nhidden = fparams['nHIDDEN']
    nclasses = fparams['nCLASSES']
    batchnorm = fparams['USEBATCHNORM']
    use_penalty = fparams['USE_PENALTY']
    
    ini = lasagne.init.GlorotUniform()
    nonlin = lasagne.nonlinearities.rectify
    linear = lasagne.nonlinearities.linear
    softmax = lasagne.nonlinearities.softmax

    
    # Input Layer
    l_in = lasagne.layers.InputLayer(shape=input_shape, input_var=input_var)
    
    l_focus1 = FocusedLayer1D(l_in, num_units=nhidden, 
                              nonlinearity=linear, name='focus1',
                              trainMus=fparams['trainMus'],
                              trainSis=fparams['trainSis'],
                              initMu=fparams['initMu'], 
                              W=ini, withWeights=fparams['withWeights'], 
                              bias=lasagne.init.Constant(0.0), 
                              initSigma=fparams['initSigma'], 
                              trainWs=fparams['trainWs'])
                                               
    
    if batchnorm:
    # if you close BATCHNORM weights get LARGE
        l_bn = lasagne.layers.NonlinearityLayer(
                lasagne.layers.BatchNormLayer(l_focus1), nonlinearity=nonlin)
    
    else:
    
        l_bn = lasagne.layers.NonlinearityLayer(l_focus1, nonlinearity=nonlin)
 
    #l_drop1 = lasagne.layers.dropout(l_bn, p=0.1)
    
    # Output
    l_out = lasagne.layers.DenseLayer(l_bn, num_units=nclasses, 
                                      nonlinearity=softmax, W=ini, name='output')
    
    penalty = l2(l_out.W)*np.float32(1e-5)
    # if focusing neuron has weights. add penalty to them as well. A 10th factors less because focus function already suppresses weghts
    if fparams['withWeights']:
        penalty += l2(l_focus1.W)*np.float32(1e-5)
    
    if not use_penalty:
        penalty = penalty*0
    
    
    return l_out, penalty

In [ ]:
# Compile train and eval functions
def build_functions(using_model, X, y, fparams, penalty):
    

    # training output
    output_train = lasagne.layers.get_output(using_model, X, deterministic=False)

    # evaluation output. Also includes output of transform for plotting
    output_eval = lasagne.layers.get_output(using_model, X, deterministic=True)
    
    cost_train = T.mean(lasagne.objectives.categorical_crossentropy(output_train, y)) + penalty # Regularization
    cost_eval = T.mean(lasagne.objectives.categorical_crossentropy(output_eval, y))
  
    network_params = lasagne.layers.get_all_params(using_model, trainable=True)
    print("Params",network_params)
    n_params =  lasagne.layers.count_params(using_model)
    print("Param count:",n_params)
    
    LR_rate = theano.shared(np.float32(fparams['LEARNING_RATE']),name='lr_all')
    LR_MU = theano.shared(np.float32(fparams['LR_MU']),name='lr_mu')
    LR_SI = theano.shared(np.float32(fparams['LR_SI']),name='lr_si')
    LR_FW = theano.shared(np.float32(fparams['LR_FW']),name='lr_fw')
    LR_params = [LR_rate, LR_MU, LR_SI, LR_FW]
        
    updates = sgdWithLrsClip(cost_train, network_params, learning_rate=LR_rate, 
                         mu_lr=LR_MU, si_lr=LR_SI, 
                         focused_w_lr=LR_FW, momentum=fparams['MOMENTUM'])
    
    test_acc = T.mean(T.eq(T.argmax(output_eval, axis=1), y), dtype=theano.config.floatX)

    trainf = theano.function([X, y], [cost_train, output_train, penalty], updates=updates, allow_input_downcast=True, name='train')
    evalf = theano.function([X, y], [cost_eval, output_eval, test_acc], allow_input_downcast=True, name='eval')
    
    
    return trainf, evalf, n_params

In [ ]:
# Epoch tranining with mini-batch
def epoch_func(X, y, fnc, penalty=False, batch_size=512):
    num_samples = X.shape[0]
    num_batches = int(np.ceil(num_samples / float(batch_size)))
    costs = []
    correct = 0
    penalties = []
    for i in range(num_batches):
        idx = range(i*batch_size, np.minimum((i+1)*batch_size, num_samples))
        X_batch = X[idx]
        y_batch = y[idx]
        outputs = fnc(X_batch, y_batch)
        costs += [outputs[0]]
        probs = outputs[1]
        preds = np.argmax(probs, axis=1)
        correct += np.sum(y_batch == preds)
        #print("corr:",correct)
        if fnc.name.startswith('train'):
            if penalty:
                penalties += [outputs[2]]
                
    acc = correct/float(num_samples)
    pen = []
    if len(penalties)>0:
        pen = np.mean(penalties)
    
    return np.mean(costs), acc , pen 

## EXPERIMENT SETTINGS 

In [ ]:
# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "bilisim/"
EXPERIMENT_ID = "ex-2/"
RANDSEED = 10
paper_fig_settings()

from  datetime import datetime
now = datetime.now()
timestr = now.strftime("%Y%m%d-%H%M%S")
logdir =  os.path.join(PROJECT_ROOT_DIR, "outputs", 
                        CHAPTER_ID+EXPERIMENT_ID)


lasagne.random.set_rng(np.random.RandomState(RANDSEED))  # Set random state so we can investigate results
np.random.seed(RANDSEED)
np.set_printoptions(formatter={'float': '{: 0.4f}'.format}, suppress=True)

PLOT_INT_RESULTS = False
PRINT_INTERVAL = 10
RECORD_INTERVAL= 1



In [ ]:
print(CHAPTER_ID+EXPERIMENT_ID)

In [ ]:
# config settingw
Focusing_layer_params={}
Focusing_layer_params['trainSis'] = True
Focusing_layer_params['trainMus'] = True    
Focusing_layer_params['trainWs'] = True
Focusing_layer_params['withWeights'] = True
Focusing_layer_params['initMu'] = 'spread'# 'middle' Or sth else
Focusing_layer_params['initSigma'] = 0.1
print(Focusing_layer_params)

                              

In [ ]:
# Hyper Parameter Test SETS
NUM_EPOCHS = 500
dset_choice = 'Synt' #'Mnist','Cifar10'
#dset_choice = 'Mnist'
Synthetic_Data_Params={}
if dset_choice=='Synt':

    Synthetic_Data_Params['nSAMPLES'] = 5000
    Synthetic_Data_Params['nCLASSES'] = 4
    #nFEATURES= np.array([4,10,40,100,1000])
    Synthetic_Data_Params['nFEATURES']= 20
    Synthetic_Data_Params['nCLUSTERS'] = 1
    Synthetic_Data_Params['NOISE_DIMS'] = 0.5 # We add this many NOISE_DIMS*nFeatures to nFeatures
    Synthetic_Data_Params['NOISE_SCALE'] = 1.0 # data is standard, this should not affect anything
    Synthetic_Data_Params['NOISE_PATTERN']= 'sides' # position of the noise with respect to sampled dataset.

Training_Params = {}
Training_Params['BATCH_SIZE'] = 0.05 # portion of training data
Training_Params['MOMENTUM'] = np.float32(0.9) # .9 is slightly worse
Training_Params['LEARNING_RATE'] = np.float32(0.01) # works good. 0.001 is too low for many datasets. 

Network_Params = {}
Network_Params['USEBATCHNORM'] = False #False is 2x 3x times faster, and performance is worse
Network_Params['USE_PENALTY'] = False  # No significant difference perhaps due to low number of epochs
Network_Params['nHIDDEN'] = 10


# this method chooses random values from a list of arrays
def choose_rand(x):
    val=[]
    for k in range(len(x)):
        v = np.random.choice(x[k])
        
        val.append(v)
    return val


def set_lr_rates(d):
    lr_all =d['LEARNING_RATE']    
    lr_fw = lr_all
    INIT_SIGMA = d['initSigma']
    lr_tr_coeff = 39.08*np.exp(-20.65*INIT_SIGMA)+1.16
    #lr_tr_coeff = 1.0
    print(lr_tr_coeff)
    lr_mu = lr_fw/lr_tr_coeff
    lr_si = lr_mu*0.82
    d['LR_MU']= np.float32(lr_mu)
    d['LR_SI'] = np.float32(lr_si)
    d['LR_FW'] = np.float32(lr_all)
    
# combine parameters

pars ={}
pars.update(Synthetic_Data_Params)
pars.update(Training_Params)
pars.update(Network_Params)
pars.update(Focusing_layer_params)
set_lr_rates(pars)


print(pars)


In [ ]:
# prepare data set here. 
if dset_choice=='Synt':
    
    #set batch size so that it is reasonably smaller than nSAMPLES# larger batch sizes will require different LR
    pars['BATCH_SIZE']=np.int(pars['BATCH_SIZE']*pars['nSAMPLES'])

    # it is not possible to sample less dimensional data than nclusters*nclasses
    pars['nFEATURES'] = max(pars['nFEATURES'],pars['nCLUSTERS']*pars['nCLASSES'])
    if 2**pars['nFEATURES'] < pars['nCLUSTERS']*pars['nCLASSES']:
        pars['nFEATURES'] = (pars['nCLUSTERS']*pars['nCLASSES'])+1
    
    print("Feature num check", pars['nFEATURES']>=pars['nCLUSTERS']*pars['nCLASSES'])

    noisy_features = int(pars['NOISE_DIMS']*pars['nFEATURES'])
    
    # at each repeat we create a train/test data first sample data 
    data = load_blob(classes=pars['nCLASSES'], features=pars['nFEATURES'], samples=pars['nSAMPLES'],
                     random_state=RANDSEED, noise_dims=noisy_features,
                     noise_scale=1.0, noise_pattern=pars['NOISE_PATTERN'],clusters=pars['nCLUSTERS'])

    n_features = data['X_train'].shape[1]
    X_train, y_train, X_test, y_test = data['X_train'], data['y_train'], data['X_test'],data['y_test']
    X = T.fmatrix()
    y = T.ivector('targets')
    input_shape= (X_train.shape[0],X_train.shape[1])
elif dset_choice=='Mnist':
    data_root = '../datasets/image/'
    X_train, y_train, X_val, y_val, X_test, y_test = load_dataset_mnist(folder=data_root)
    n_features = X_train.shape[1] 
    pars['BATCH_SIZE'] = 512
    pars['nCLASSES'] = 10
    X = T.tensor4('inputs')
    input_shape= (None, X_train.shape[1],X_train.shape[2],X_train.shape[3])
    y = T.ivector('targets')
    
print(pars)
print("Train data: ",X_train.shape,y_train.shape )
print("Test data: ",X_test.shape,y_test.shape )

In [ ]:
model_dense, penalty_dense = build_dense_model(input_shape,X, pars)
mp_dense = lasagne.layers.get_all_params(model_dense, trainable=True)


model_focus, penalty_focus = build_focused_model(input_shape,X,  pars)
mp_focus = lasagne.layers.get_all_params(model_focus, trainable=True)
l_focused = next(l for l in lasagne.layers.get_all_layers(model_focus) if l.name is 'focus1')


train_dense, eval_dense,n_netparamsd = build_functions(model_dense,X,y, pars,  penalty_dense)
train_focus, eval_focus,n_netparamsf = build_functions(model_focus,X,y, pars, penalty_focus)
pars['n_netparamsd'] = n_netparamsd
pars['n_netparamsf'] = n_netparamsf

print(pars)

In [ ]:
# create utility functions

l_dense = next(l for l in lasagne.layers.get_all_layers(model_dense) if l.name is "dense1")
dense_weights = l_dense.W

l_focused = next(l for l in lasagne.layers.get_all_layers(model_focus) if l.name is 'focus1')

# create functions to read mu, si, and weight values.
get_si = lambda: l_focused.si.get_value()
get_mu = lambda: l_focused.mu.get_value()
get_foci = lambda: l_focused.calc_u().eval().T
get_w = lambda: l_focused.W.get_value()
set_w = lambda value: l_focused.W.set_value(value)


# create output functions to read output of focus layer

output_focus = lasagne.layers.get_output(l_focused, X, deterministic=True)
eval_output_focus = theano.function([X], [output_focus], allow_input_downcast=True)
    
output_dense = lasagne.layers.get_output(l_dense, X, deterministic=True)
eval_output_dense = theano.function([X], [output_dense], allow_input_downcast=True)

In [ ]:
# now do the training and test then record the outputs

total_time_focus = 0
total_time_dense = 0
costs_dense, costs_focus = [], []
costs_tst_dense, costs_tst_focus = [], []
accs_dense, accs_focus = [], []

focus_outputs = []
dense_weights = []
mus = []
sis = []
foci = []
w_change = []
scalers = []
#np.random.shuffle(X_test)

test_subset = X_test[np.random.permutation(X_test.shape[0])[0:500],:]
try:
    for n in range(NUM_EPOCHS):
        
        
        if n == 0 or n%RECORD_INTERVAL==0: 
            mus.append(get_mu())
            sis.append(get_si())
            foci.append(get_foci()*get_w())
            w_change.append(get_w())
            
            focus_output = eval_output_focus(test_subset)
            focus_outputs.append(focus_output)
            
        #print("w:",np.asarray(get_w())[:,0], "mu: ",get_mu()[0],"si: ",get_si()[0])
        start_time = time.time()
        train_cost_dense, train_acc_dense, penalty_dense = epoch_func(X_train, 
                                                                      y_train, 
                                                                      train_dense,
                                                                      penalty=pars['USE_PENALTY'],
                                                                      batch_size=pars['BATCH_SIZE'])
        time_spent_dense = time.time() - start_time
        total_time_dense +=time_spent_dense
        
        start_time = time.time()
        train_cost_focus, train_acc_focus, penalty_focus = epoch_func(X_train, 
                                                                      y_train, 
                                                                      train_focus,
                                                                      penalty=pars['USE_PENALTY'],
                                                                      batch_size=pars['BATCH_SIZE'])
        time_spent_focus = time.time() - start_time
        total_time_focus +=time_spent_focus
    
        tst_cost_dense, acc_dense,_= epoch_func(X_test, y_test, eval_dense)
        tst_cost_focus, acc_focus,_ = epoch_func(X_test, y_test, eval_focus)
        
        if not penalty_dense:
            penalty_dense = 0 
        if not penalty_focus:
            penalty_focus = 0 
        if np.mod(n, PRINT_INTERVAL) == 0:
            print(("Dense Ep {0}: Tcst {1:1.5f}, Tacc {2:1.3f}, " +
            "Vcst {3:1.5f}, Vacc {4:3.3f}, Pen: {5:1.5f}," +
            "Time: {6:1.4f}").format(n, train_cost_dense, train_acc_dense,tst_cost_dense,acc_dense, penalty_dense, time_spent_dense))
            print(("Focus Ep {0}: Tcst {1:1.5f}, Tacc {2:1.3f}, " +
            "Vcst {3:1.5f}, Vacc {4:3.3f}, Pen: {5:1.5f}," +
            "Time: {6:1.4f}").format(n, train_cost_focus, train_acc_focus,tst_cost_focus,acc_focus, penalty_focus,time_spent_focus))
            
            
        costs_dense.append(train_cost_dense)
        costs_focus.append(train_cost_focus)
    
        costs_tst_dense.append(tst_cost_dense)
        costs_tst_focus.append(tst_cost_focus)
    
        accs_focus.append(acc_focus)
        accs_dense.append(acc_dense)
        
except KeyboardInterrupt:
    pass

acc_focus_np = np.array(accs_focus)
acc_dense_np = np.array(accs_dense)

In [ ]:
from plot_utils import save_fig, paper_fig_settings

In [ ]:
SMALL_SIZE = 14
MEDIUM_SIZE = 18
BIGGER_SIZE = 22

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=SMALL_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title


In [ ]:
plt.figure(figsize=(6,6))
plt.title(u"Loss")
plt.plot(np.array(costs_focus), linestyle='--',color='r', linewidth=2)
plt.plot(np.array(costs_dense), linestyle='--', color='g', linewidth=2)
plt.plot(np.array(costs_tst_focus), color='r', linewidth=4)
plt.plot(np.array(costs_tst_dense), color='g', linewidth=4)

#plt.yscale('log')
#plt.grid(True, which='both')
plt.legend([u'Fcs-trn', u'Dns-trn', u'Fcs-tst',u'Dns-tst'])
plt.grid(True)
#plt.xlabel('Epoch')
plt.xlabel('Epok')
#plt.ylabel('Error')
plt.ylabel('Hata')
plt.xlim(0,len(costs_focus))
#save_fig('focus_train')
save_fig('training_val_loss_synthetic', postfix=CHAPTER_ID+EXPERIMENT_ID)
plt.show()


In [ ]:
acc_focus_np = np.array(accs_focus)
acc_dense_np = np.array(accs_dense)
plt.figure(figsize=(6,6))
#plt.title(u'Doğrulama Kümesi Doğruluk')
plt.plot(np.array(acc_focus_np), color='r', linewidth=3, alpha=0.5)
plt.plot(np.array(acc_dense_np), color='g', linewidth=3, alpha=0.5)
#plt.legend([u'Odaklanan Nöron Katmanlı Sinir Ağı', u'Tümden Bağlı Katmanli Yapay Sinir Ağı'])
plt.legend([u'Focused', u'Dense'])
#save_fig('focus_acc')
plt.xlabel('Epok')
plt.ylabel(u'Sınıflandırma Doğruluğu (Acc)')
plt.grid(True)
plt.ylim([0.5,1.0])
save_fig('val_accuracy_synth',postfix=CHAPTER_ID+EXPERIMENT_ID)
plt.show()
#savefig(fname, dpi=None, facecolor='w', edgecolor='w',
#          orientation='portrait', papertype=None, format=None,
#          transparent=False, bbox_inches=None, pad_inches=0.1,
#          frameon=None, metadata=None)


In [ ]:
focus_outputs_np = np.array(focus_output).reshape((-1, pars['nHIDDEN']))
for i in range(3):
    x = focus_outputs_np[:, i]
    n, bins, patches = plt.hist(x, 50, normed=0, facecolor='green', alpha=0.75)

    plt.xlabel('Output Activation Distribution')
    plt.ylabel('Count')
    plt.title(r'$\mathrm{Histogram\ of\ focus\ Neuron:}\ '+ str(i) +'$')
    plt.grid(True)
    plt.show()

In [ ]:
import itertools
markers = itertools.cycle((',', '+', '.', '^', '*')) 
styler = itertools.cycle(('-', '--', '-.')) 

mu = np.array(mus)
print (mu.shape)

plt.figure(figsize=(4, 4))
plt.title(r'Foci center change')
plt.xlabel('Epoch')
plt.ylabel(r'$\mu$')
plt.ylim([0.2,0.8])

#plt.title(r'$\mu$' + u' Değisimi')
#plt.xlabel('Epok')
#plt.ylabel(r'$\mu$')

plt.plot(mu[:, 0] , marker='.')
for x in range(mu.shape[1]):
    plt.plot(mu[0:-1, x], marker='.')
plt.grid(True)
#plt.xlim((-100,NUM_EPOCHS))
#plt.xscale('log')
save_fig('mu_change',postfix=CHAPTER_ID+EXPERIMENT_ID)
plt.show()


In [ ]:
si = np.array(sis)

plt.figure(figsize=(4, 4))
plt.title(r'$\sigma$' + u' Change')


plt.xlabel('Epoch')
plt.ylabel(r'$\sigma$')

#plt.plot(mu[:, 0] * 16, marker='o')
for x in range(si.shape[1]):
    plt.plot(si[0:-1, x], marker='.')
plt.grid(True)
#plt.xlim((-100,NUM_EPOCHS))
#plt.xscale('log')
save_fig('si_change', postfix=CHAPTER_ID+EXPERIMENT_ID)
plt.show()

